## Домашнее задание

In [1]:
import numpy as np
import torch
import torch.nn as nn

## Task 1

Написать на PyTorch глубокую сеть. Проверить работу форвард пасса.

In [2]:
# Генерация бачей.
samples = np.random.uniform(low=0.0, high=1000.0, size=(15,5))

batch = torch.Tensor(samples)
print(batch, batch.size(), batch.dim(), batch.dtype)

tensor([[116.1369, 746.0771, 519.6165, 716.3768,  32.1776],
        [496.6722, 599.8477, 457.7296, 789.0613, 622.1443],
        [142.0859,  25.7229, 404.7177, 595.1578, 489.3096],
        [178.1713, 769.6541, 483.3534, 973.6365, 767.7343],
        [967.2346, 956.6804, 243.5869, 964.2245, 261.6455],
        [396.1648, 368.6633, 739.4695, 515.0351, 775.8729],
        [372.0689, 514.3693, 677.1927, 388.8015, 626.9980],
        [910.5991, 746.9720, 443.7657, 555.8637, 897.4891],
        [848.0792, 490.1591, 460.3412, 651.9518, 448.3521],
        [709.1456, 767.9954,  37.3802, 804.2241, 674.7280],
        [387.8181, 976.7886, 158.6455, 850.2786, 478.3173],
        [515.4023, 177.1944,  99.1818, 173.9888,   4.5867],
        [629.2847, 361.1377, 303.4508, 715.5740, 636.0769],
        [ 66.6064, 173.3002, 950.3172, 464.0774, 479.8605],
        [565.1191, 339.9091, 566.9595, 318.1571,  10.6617]]) torch.Size([15, 5]) 2 torch.float32


<img src="./data/Deep%20Convolutional%20Network%20%28DCN%29.png" width="640"/>

In [3]:
class TestModel(nn.Module):
    """
    Deep Convolutional Network (DCN)
    """
    def __init__(self,):
        super().__init__()
        self.layer1 = nn.Linear(5, 5)
        self.activation1 = nn.Sigmoid()
        self.layer2 = nn.Linear(5, 4)
        self.activation2 = nn.Sigmoid()
        self.layer3 = nn.Linear(4, 3)
        self.activation3 = nn.Sigmoid()
        self.layer4 = nn.Linear(3, 2)
        self.activation4 = nn.Sigmoid()
        self.layer5 = nn.Linear(2, 4)
        self.activation5 = nn.ReLU()
        self.layer6 = nn.Linear(4, 4)
        self.activation6 = nn.ReLU()
        self.layer7 = nn.Linear(4, 3)
        
    def forward(self, x):
        x = self.activation1(self.layer1(x))
        x = self.activation2(self.layer2(x))
        x = self.activation3(self.layer3(x))
        x = self.activation4(self.layer4(x))
        x = self.activation5(self.layer5(x))
        x = self.activation6(self.layer6(x))
        x = self.layer7(x)
        return x

model = TestModel()
model.train()
model(batch)

tensor([[-0.0263,  0.1230, -0.2510],
        [-0.0263,  0.1230, -0.2510],
        [-0.0263,  0.1231, -0.2510],
        [-0.0263,  0.1230, -0.2510],
        [-0.0263,  0.1230, -0.2510],
        [-0.0263,  0.1231, -0.2510],
        [-0.0263,  0.1231, -0.2510],
        [-0.0263,  0.1231, -0.2510],
        [-0.0263,  0.1231, -0.2510],
        [-0.0263,  0.1230, -0.2510],
        [-0.0263,  0.1230, -0.2510],
        [-0.0263,  0.1231, -0.2510],
        [-0.0263,  0.1231, -0.2510],
        [-0.0264,  0.1231, -0.2510],
        [-0.0263,  0.1231, -0.2510]], grad_fn=<AddmmBackward0>)

## Task 2

Написать адаптивный оптимизатор.

**Оптимизатор. RMSprop**  

```
accumulated += rho (0.9-0.99) * accumulated + (1 - rho) * gradient ** 2
adapt_lr = lr / sqrt(accumulated)
w = w — adapt_lr * gradient
```

In [4]:
class RMSprop:
    """
    accumulated += rho (0.9-0.99) * accumulated + (1 - rho) * gradient ** 2
    adapt_lr = lr / sqrt(accumulated)
    w = w — adapt_lr * gradient
    """
    def __init__(self, model: nn.Linear, lr=0.001, rho=0.99, momentum=0.99):
        self.model = model
        self.lr = lr
        self.rho = rho
        self.acc_w = np.zeros_like(model.w)
        self.acc_b = np.zeros_like(model.b)

    def step(self):
        self.acc_w += self.rho * self.acc_w + (1.0 - self.rho) * (self.model.d_w ** 2)
        self.acc_b += self.rho * self.acc_b + (1.0 - self.rho) * (self.model.d_b ** 2)
        
        adapt_lr_w = self.lr / sqrt(self.acc_w)
        adapt_lr_b = self.lr / sqrt(self.acc_b)
        
        self.model.w = self.model.w - adapt_lr_w * self.model.d_w
        self.model.b = self.model.b - adapt_lr_b * self.model.d_b

    def zero_grad(self):
        self.model.d_w = np.zeros_like(self.model.d_w)
        self.model.d_b = np.zeros_like(self.model.d_b)

## Task 3

Решить задачу нахождения корней квадратного уравнения методом градиентного спуска.

Ищем корни уравнения $x^2 - 6 \cdot x + 4 = 0$

In [5]:
a = 1
b = -6
c = 4

def grad(x):
    # Производная для уравнения (a * x^2 + b*x + c)^2
    return 2 * (a * (x ** 2) + b * x + c) * (2 * a * x + b)
    

def find_x1_root(x_start, lr=1e-3, precision=1e-5, step=1000):
    state_found = False
    n_steps = 0
    x = x_start
    for i in range(step):
        x_curr = x
        x = x - lr * grad(x)

        n_steps = i + 1

        if abs(x - x_curr) < precision:
            state_found = True
            break

    if state_found:
        print(f'минимум {x:.3f} найден: x_start={x_start}, lr={lr}, steps={n_steps}')
    else:
        print(f'минимум НЕ найден: x_start={x_start}, lr={lr}, steps={n_steps}')
        
    return x

def num_roots():
    d = b^2 - 4*a*c
    if d > 0:
        return 2
    elif d == 0:
        return 1
    return 0

def find_x2_root(x1):
    # Второй корень ищется по теореме Виета (если уравнение имеет 2-а различных корня)
    return -(b/a) - x1

print(f'Количество корней: {num_roots()}')

find_x1_root(0, 1e-1)
x1 = find_x1_root(0, 1e-2)
find_x1_root(0, 1e-3)
find_x1_root(5, 1e-2)
find_x1_root(6, 1e-2)
find_x1_root(10, 1e-4)

# В данном случае уравнение имеет 2-а различных корня
print(f'Второй корень уравнения: {find_x2_root(x1):.3f}')

Количество корней: 2
минимум НЕ найден: x_start=0, lr=0.1, steps=1000
минимум 0.764 найден: x_start=0, lr=0.01, steps=20
минимум 0.764 найден: x_start=0, lr=0.001, steps=187
минимум 5.236 найден: x_start=5, lr=0.01, steps=20
минимум 5.236 найден: x_start=6, lr=0.01, steps=20
минимум НЕ найден: x_start=10, lr=0.0001, steps=1000
Второй корень уравнения: 5.236


**Вывод:**

1. Всегда ли сойдемся за приемлемое количество шагов?  
Если правильно выбрать начальную точку и лернинг рейт, то можно сойтись за приемлемое количество шагов.

2. Важна ли начальная точка?  
Важно чтобы начальная точка была ближе к минимуму.

3. Как найти второй корень?  
Второй корень ищется по теореме Виета (если уравнение имеет 2-а различных корня).

4. Как вляет ЛР?  
Да, лернинг рейт влияет на скорость приближения к минимуму.